In [1]:
import copy
from queue import PriorityQueue

### utils

In [2]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)


def reconstruct_path(came_from, current):
    #     print('reconstruct_path start from:{}'.format(current.get_pos()))
    path = [current.get_pos()]
    while current in came_from:
        current = came_from[current]
        if not isinstance(current, str):
            current.make_path()
            path.append(current.get_pos())
        else:
            path.append(came_from[current].get_pos())
    return path


def get_turn_point(turn_val):
    turn_pos = turn_val.split('_')
    gap = settings.WIDTH // settings.ROWS
    return Spot(int(turn_pos[1]), int(turn_pos[2]), gap, settings.ROWS)


### came_from이 쌓임 (3, 5):(3, 4), (2, 5):(3, 5), (1, 5):(2, 5)
### came_from은 어디서왔는지를 나타냄, 타입은 딕셔너리, neighbor, 현재위치 (갈수있는 neighbor들의 위치값이 저장되어있음)
### (neighbor,curret위치)

def count_reconstruct_path(came_from, current):
    cnt = 0
    while current in came_from:
        current = came_from[current]
        cnt += 1
    return cnt


def check_turn(came_from, current):
    ### return False if position didn't move for more than 2
    if count_reconstruct_path(came_from, current) < 2:
        return False
    ### check if current position is already turning (by checking if value of current is string instead of Spot)
    if isinstance(came_from[current], str):
        #         print('@@ check_turn already turning!')
        return False
    ### 전전 위치가 회전중(string 타입)이라면, 그 전꺼의 위치로 회전 여부 파악
    ### ex) 현재위치: (4, 0),  경로: (3, 1), 'turn_4_1', (4, 1), (4, 0) -> 경로를 다음과 같이 처리: (3, 1), (4, 1), (4, 0)
    elif isinstance(came_from[came_from[current]], str):
        last_last_pos = came_from[came_from[came_from[current]]].get_pos()
    else:
        last_last_pos = came_from[came_from[current]].get_pos()

    cur_pos = current.get_pos()
    last_pos = came_from[current].get_pos()
    if last_last_pos[0] == last_pos[0] and cur_pos[0] != last_pos[0] or \
            last_last_pos[1] == last_pos[1] and cur_pos[1] != last_pos[1]:
        return True
    else:
        return False


def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, rows - j - 1, gap, rows)
            grid[i].append(spot)

    return grid


def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos

    row = y // gap
    col = x // gap

    return row, col


def add_obstacle(grid, x, y):
    spot = grid[x][y]
    spot.make_barrier()
    spot.update_neighbors(grid)
    for row in grid:
        for s in row:
            s.update_neighbors(grid)




def remove_obstacle(grid, x, y):
    spot = grid[x][y]
    spot.reset()
    spot.update_neighbors(grid)
    for row in grid:
        for s in row:
            s.update_neighbors(grid)


def setRack(grid, rack_list):
    for rack_point in rack_list:
        spot = grid[rack_point[0]][rack_point[1]]
        spot.make_barrier()
    for row in grid:
        for s in row:
            s.update_neighbors(grid)


def comparePosition(pos1, pos2):
    return (pos1[0] == pos2[0]) and (pos1[1] == pos2[1])


def setAGVObstacle(tick, grid, agv_list):
    for agv in agv_list:
        start_tick = agv['start_tick']
        agv_path = agv['path']
        tick_interval = tick - start_tick

        if tick_interval >= 0 and tick_interval < len(agv_path):
            add_obstacle(grid, agv_path[tick_interval][0], agv_path[tick_interval][1])
            if tick_interval > 0 and comparePosition(agv_path[tick_interval], agv_path[tick_interval - 1]):
                return
            elif tick_interval > 0:
                remove_obstacle(grid, agv_path[tick_interval - 1][0], agv_path[tick_interval - 1][1])
        ### TODO: current가 turn을 해서 tick 이 +2 될 경우 이전 obstacle을 삭제하지 못하는 경우 발생!!!

def detect_collision(tick, current, neighbour, agv_list):
    for agv in agv_list:
        start_tick = agv['start_tick']
        agv_path = agv['path']
        tick_interval = tick - start_tick
        if tick_interval <= 0:
            continue

        ### 1. tick에 현재 agv 위치(current) vs tick+1에 obstacle(agv_path[tick]) 위치 비교
        ### 2. tick+1에 현재 agv 위치(neighbour) vs tick에 obstacle(agv_path[tick+1]) 위치 비교
        if current.get_pos() == agv_path[tick_interval] and neighbour.get_pos() == agv_path[tick_interval+1]:
            return True ### detected collision
    return False ### there is no collision



# def check_throughpass(tick, current, came_from, agv):
#     start_tick = agv['start_tick']
#     agv_path = agv['path']
#     tick_interval = tick - start_tick + 1
#     if tick_interval >= 0 and tick_interval < len(agv_path):

### Spot

In [3]:
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)


class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = ROWS - col - 1
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == RED

    def is_open(self):
        return self.color == GREEN

    def is_barrier(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == TURQUOISE

    def reset(self):
        self.color = WHITE

    def make_start(self):
        self.color = ORANGE

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK

    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE

    # def draw(self, win):
    #     pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        ## 색깔 + 장애물 (False)
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False


### CAstar algorithm

In [4]:
def algorithm(grid, start, end, start_tick, AGV1_PATH):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    open_set_hash = {start}

    ##openset이 있을 때 동안 계속 실행
    while not open_set.empty():
        # prev_open_set[2]는 spot을 의미한다.
        prev_open_set = open_set.get()

        current = prev_open_set[2]
        # Camefrom은 어디서왔는지를 나타냄, 타입은 딕셔너리, neighbor, 현재위치 (갈수있는 neighbor들의 위치값이 저장되어있음)
        # (neighbor,Curret위치)

        # tick을 계산하기 위해 count_reconstruct_path함수를 실행
        tick = count_reconstruct_path(came_from, current) + start_tick
        print('@@@ tick:{} current:{}'.format(tick, current.get_pos()))

        ### 장애물을 추가하는 함수
        setAGVObstacle(tick, grid, AGV1_PATH)
        current.update_neighbors(grid)

        ### open_set이랑 똑같은데 spot만 가진다(위치)
        open_set_hash.remove(current)

        if current == end:
            path = reconstruct_path(came_from, end)
            path.reverse()
            return path

        for neighbor in current.neighbors:
            temp_g_score = g_score[current]

            if temp_g_score < g_score[neighbor]:
                if check_turn(came_from, current):
                    ## check if current selected 'current' is a turn
                    ### current: 2,5    neighbor: 1,5     came_from[current]: 3,5
                    ### 기존 came_from: (3, 5):(3, 4), (2, 5):(3, 5), (1, 5):(2, 5)
                    ### 바꾼 came_from: (3, 5):(3, 4), (turn):(3, 5), (2, 5):(turn), (1, 5):(2, 5)
                    came_from['turn_{}_{}'.format(came_from[current].get_pos()[0],
                                                  came_from[current].get_pos()[1])] = came_from[current]
                    came_from[current] = 'turn_{}_{}'.format(came_from[current].get_pos()[0],
                                                             came_from[current].get_pos()[1])
                came_from[neighbor] = current

                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())

                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        if current != start:
            current.make_closed()

    return False


In [5]:
RACK_LOC = [(1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
            (2, 2), (2, 3), (2, 4), (2, 5), (2, 6),
            (5, 2), (5, 3), (5, 4), (5, 5), (5, 6),
            (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)]
ROWS = 8
WIDTH = 300
AGV1_PATH = {"start_tick":1, "path":[(2,0), (2,1), (1,1), (0,1), (0,2), (0,2), (0,2), (0,1), (0,0)]}
AGV2_PATH = {"start_tick":3, "path":[(3, 0), (4, 0), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4)]}

AGV_LIST = [AGV1_PATH, AGV2_PATH]

if __name__ == '__main__':
    grid = make_grid(ROWS, WIDTH)
    setRack(grid, RACK_LOC)

    # start1 = grid[3][0]
    # end1 = grid[7][6]
    # start2 = grid[5][0]
    # end2 = grid[3][3]
    start3 = grid[0][3]
    end3 = grid[4][4]

    AGV3 = algorithm(grid, start3, end3, 1, AGV_LIST)
    print('@@@@@@@@@@algorithm result:{}'.format(AGV3))


@@@ tick:1 current:(0, 3)
@@@ tick:2 current:(0, 4)
@@@ tick:3 current:(0, 5)
@@@ tick:2 current:(0, 2)
@@@ tick:4 current:(0, 6)
@@@ tick:3 current:(0, 1)
@@@ tick:5 current:(0, 7)
@@@ tick:6 current:(1, 7)
@@@ tick:8 current:(2, 7)
@@@ tick:9 current:(3, 7)
@@@ tick:10 current:(4, 7)
@@@ tick:11 current:(4, 6)
@@@ tick:13 current:(4, 5)
@@@ tick:14 current:(4, 4)
@@@@@@@@@@algorithm result:[(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (4, 7), (4, 6), (4, 5), (4, 4)]


In [6]:
grid[0][0]

In [7]:
print(len(grid))

8
